In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.initializers import VarianceScaling


In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
cd drive/'My Drive'/'Colab Notebooks'/test

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/test'
/content/drive/My Drive/Colab Notebooks/test


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 24
compression = 0.5
dropout_rate = 0.2
weight_decay = 1E-4 # It is used in l2 regularisation, this specific value is most commonly used

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

` VarianceScaling(scale = 2.0, mode = 'fan_in', distribution = 'normal', seed = None)` is used for MSRA initialization as per the dense net paper.
Dropout is used to solve overfitting along with it l2 regulirisation is used to increase `val_acc` 

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization(axis = -1, gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D( num_filter, (3,3), kernel_initializer = VarianceScaling(scale = 2.0, mode = 'fan_in', distribution = 'normal', seed = None), use_bias=False ,padding='same' )(relu) # msra initialization
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization(axis = -1, gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), kernel_initializer = VarianceScaling(scale = 2.0, mode = 'fan_in', distribution = 'normal', seed = None), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization(axis = -1, gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 16 # To get as many major features as possible the number of filter  is more (16)
dropout_rate = 0.2
l = 18
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), kernel_initializer = VarianceScaling(scale = 2.0, mode = 'fan_in', distribution = 'normal', seed = None), use_bias=False ,padding='same')(input)

num_filter = 12

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [17]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
data_generator = datagen.flow(x_train, y_train, batch_size=64)
test_generator = ImageDataGenerator().flow(x_test, y_test, batch_size=64)

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True) # initially these values are set based on the Densenet paper
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

*Initially we start with low batch size as it introduces stochoistic noise which helps to generalize the model and initial lr is high so that the `val_acc` does not sit on the floor.(As per the paper "[A walk  through SGD](https://arxiv.org/pdf/1802.08770.pdf)")*

In [0]:
filepath= "weights.{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=2,
                    verbose=1,
                    validation_data=(x_test, y_test))

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs=62,
                    verbose=1,
                    initial_epoch = 2,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

In [0]:
model.load_weights('weights.62-0.80.hdf5')

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.07, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=65,
                    verbose=1,
                    initial_epoch=62,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.05, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs=80,
                    verbose=1,
                    initial_epoch = 65,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

In [0]:
model.load_weights('weights.80-0.84.hdf5')

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.04, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 100,
                    verbose=1,
                    initial_epoch = 80,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 81/100


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 418s 535ms/step - loss: 0.6129 - acc: 0.8092 - val_loss: 0.8572 - val_acc: 0.7664

Epoch 00081: val_acc did not improve from 0.85567
Epoch 82/100
781/781 [==============================] - 395s 506ms/step - loss: 0.6030 - acc: 0.8117 - val_loss: 0.9659 - val_acc: 0.7430

Epoch 00082: val_acc did not improve from 0.85567
Epoch 83/100
781/781 [==============================] - 395s 506ms/step - loss: 0.5950 - acc: 0.8148 - val_loss: 0.6791 - val_acc: 0.8147

Epoch 00083: val_acc did not improve from 0.85567
Epoch 84/100
781/781 [==============================] - 395s 506ms/step - loss: 0.5882 - acc: 0.8202 - val_loss: 0.7431 - val_acc: 0.8024

Epoch 00084: val_acc did not improve from 0.85567
Epoch 85/100
781/781 [==============================] - 395s 506ms/step - loss: 0.5760 - acc: 0.8208 - val_loss: 0.7062 - val_acc: 0.8050

Epoch 00085: val_acc did not improve from 0.85567
Epoch 86/100
781/781 [==============================] - 395s 505ms/s

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.03, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 120,
                    verbose=1,
                    initial_epoch = 100,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 101/120


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 421s 539ms/step - loss: 0.5424 - acc: 0.8314 - val_loss: 0.7355 - val_acc: 0.7960

Epoch 00101: val_acc did not improve from 0.85958
Epoch 102/120
781/781 [==============================] - 395s 506ms/step - loss: 0.5411 - acc: 0.8316 - val_loss: 1.0983 - val_acc: 0.7398

Epoch 00102: val_acc did not improve from 0.85958
Epoch 103/120
781/781 [==============================] - 395s 506ms/step - loss: 0.5298 - acc: 0.8365 - val_loss: 0.7274 - val_acc: 0.8073

Epoch 00103: val_acc did not improve from 0.85958
Epoch 104/120
781/781 [==============================] - 402s 514ms/step - loss: 0.5283 - acc: 0.8382 - val_loss: 0.5938 - val_acc: 0.8401

Epoch 00104: val_acc did not improve from 0.85958
Epoch 105/120
781/781 [==============================] - 394s 505ms/step - loss: 0.5214 - acc: 0.8388 - val_loss: 0.5468 - val_acc: 0.8479

Epoch 00105: val_acc did not improve from 0.85958
Epoch 106/120
781/781 [==============================] - 395s 50

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.010, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 145,
                    verbose=1,
                    initial_epoch = 120,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 121/145


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 414s 530ms/step - loss: 0.4508 - acc: 0.8611 - val_loss: 0.4862 - val_acc: 0.8660

Epoch 00121: val_acc improved from 0.86428 to 0.86599, saving model to weights.121-0.87.hdf5
Epoch 122/145
781/781 [==============================] - 394s 504ms/step - loss: 0.4476 - acc: 0.8631 - val_loss: 0.4770 - val_acc: 0.8676

Epoch 00122: val_acc improved from 0.86599 to 0.86759, saving model to weights.122-0.87.hdf5
Epoch 123/145
781/781 [==============================] - 395s 506ms/step - loss: 0.4379 - acc: 0.8638 - val_loss: 0.5454 - val_acc: 0.8552

Epoch 00123: val_acc did not improve from 0.86759
Epoch 124/145
781/781 [==============================] - 392s 502ms/step - loss: 0.4370 - acc: 0.8669 - val_loss: 0.5486 - val_acc: 0.8561

Epoch 00124: val_acc did not improve from 0.86759
Epoch 125/145
781/781 [==============================] - 393s 503ms/step - loss: 0.4340 - acc: 0.8664 - val_loss: 0.4888 - val_acc: 0.8712

Epoch 00125: val_acc improve

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.007, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 165,
                    verbose=1,
                    initial_epoch = 145,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 146/165


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 420s 538ms/step - loss: 0.4111 - acc: 0.8718 - val_loss: 0.4918 - val_acc: 0.8682

Epoch 00146: val_acc did not improve from 0.88892
Epoch 147/165
781/781 [==============================] - 396s 506ms/step - loss: 0.4142 - acc: 0.8687 - val_loss: 0.5216 - val_acc: 0.8635

Epoch 00147: val_acc did not improve from 0.88892
Epoch 148/165
781/781 [==============================] - 396s 507ms/step - loss: 0.4140 - acc: 0.8685 - val_loss: 0.4437 - val_acc: 0.8806

Epoch 00148: val_acc did not improve from 0.88892
Epoch 149/165
781/781 [==============================] - 396s 507ms/step - loss: 0.4091 - acc: 0.8723 - val_loss: 0.4646 - val_acc: 0.8737

Epoch 00149: val_acc did not improve from 0.88892
Epoch 150/165
781/781 [==============================] - 396s 507ms/step - loss: 0.4068 - acc: 0.8726 - val_loss: 0.4283 - val_acc: 0.8807

Epoch 00150: val_acc did not improve from 0.88892
Epoch 151/165
781/781 [==============================] - 396s 50

In [0]:
model.load_weights('weights.159-0.89.hdf5')

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.004, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 172,
                    verbose=1,
                    initial_epoch = 165,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 166/172


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 425s 544ms/step - loss: 0.3914 - acc: 0.8776 - val_loss: 0.4531 - val_acc: 0.8781

Epoch 00166: val_acc improved from -inf to 0.87810, saving model to weights.166-0.88.hdf5
Epoch 167/172
781/781 [==============================] - 398s 510ms/step - loss: 0.3903 - acc: 0.8795 - val_loss: 0.4381 - val_acc: 0.8849

Epoch 00167: val_acc improved from 0.87810 to 0.88492, saving model to weights.167-0.88.hdf5
Epoch 168/172
781/781 [==============================] - 399s 511ms/step - loss: 0.3887 - acc: 0.8775 - val_loss: 0.4494 - val_acc: 0.8820

Epoch 00168: val_acc did not improve from 0.88492
Epoch 169/172
781/781 [==============================] - 398s 510ms/step - loss: 0.3860 - acc: 0.8783 - val_loss: 0.4457 - val_acc: 0.8847

Epoch 00169: val_acc did not improve from 0.88492
Epoch 170/172
781/781 [==============================] - 398s 509ms/step - loss: 0.3838 - acc: 0.8777 - val_loss: 0.4308 - val_acc: 0.8844

Epoch 00170: val_acc did not im

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.002, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 175,
                    verbose=1,
                    initial_epoch = 170,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 171/175


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 424s 543ms/step - loss: 0.3776 - acc: 0.8816 - val_loss: 0.4098 - val_acc: 0.8909

Epoch 00171: val_acc improved from 0.88532 to 0.89093, saving model to weights.171-0.89.hdf5
Epoch 172/175
781/781 [==============================] - 397s 508ms/step - loss: 0.3758 - acc: 0.8841 - val_loss: 0.4265 - val_acc: 0.8850

Epoch 00172: val_acc did not improve from 0.89093
Epoch 173/175
781/781 [==============================] - 396s 507ms/step - loss: 0.3697 - acc: 0.8858 - val_loss: 0.4057 - val_acc: 0.8905

Epoch 00173: val_acc did not improve from 0.89093
Epoch 174/175
781/781 [==============================] - 397s 508ms/step - loss: 0.3710 - acc: 0.8846 - val_loss: 0.4182 - val_acc: 0.8892

Epoch 00174: val_acc did not improve from 0.89093
Epoch 175/175
781/781 [==============================] - 398s 509ms/step - loss: 0.3730 - acc: 0.8825 - val_loss: 0.4200 - val_acc: 0.8891

Epoch 00175: val_acc did not improve from 0.89093


In [0]:
model.load_weights('weights.171-0.89.hdf5')

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.001, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 175,
                    verbose=1,
                    initial_epoch = 170,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 171/175


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 428s 548ms/step - loss: 0.3722 - acc: 0.8821 - val_loss: 0.4028 - val_acc: 0.8935

Epoch 00171: val_acc improved from 0.89093 to 0.89353, saving model to weights.171-0.89.hdf5
Epoch 172/175
781/781 [==============================] - 397s 508ms/step - loss: 0.3663 - acc: 0.8852 - val_loss: 0.4135 - val_acc: 0.8929

Epoch 00172: val_acc did not improve from 0.89353
Epoch 173/175
781/781 [==============================] - 397s 509ms/step - loss: 0.3658 - acc: 0.8853 - val_loss: 0.4156 - val_acc: 0.8909

Epoch 00173: val_acc did not improve from 0.89353
Epoch 174/175
781/781 [==============================] - 397s 509ms/step - loss: 0.3614 - acc: 0.8875 - val_loss: 0.4130 - val_acc: 0.8919

Epoch 00174: val_acc did not improve from 0.89353
Epoch 175/175
781/781 [==============================] - 397s 508ms/step - loss: 0.3667 - acc: 0.8866 - val_loss: 0.4062 - val_acc: 0.8930

Epoch 00175: val_acc did not improve from 0.89353


In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.003, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 178,
                    verbose=1,
                    initial_epoch = 175,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 176/178


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 428s 548ms/step - loss: 0.3764 - acc: 0.8825 - val_loss: 0.4536 - val_acc: 0.8817

Epoch 00176: val_acc did not improve from 0.89353
Epoch 177/178
781/781 [==============================] - 395s 505ms/step - loss: 0.3769 - acc: 0.8808 - val_loss: 0.4711 - val_acc: 0.8786

Epoch 00177: val_acc did not improve from 0.89353
Epoch 178/178
781/781 [==============================] - 394s 505ms/step - loss: 0.3776 - acc: 0.8819 - val_loss: 0.4332 - val_acc: 0.8843

Epoch 00178: val_acc did not improve from 0.89353


In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.0009, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 193,
                    verbose=1,
                    initial_epoch = 178,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 179/193


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 431s 552ms/step - loss: 0.3696 - acc: 0.8846 - val_loss: 0.4152 - val_acc: 0.8884

Epoch 00179: val_acc did not improve from 0.89353
Epoch 180/193
781/781 [==============================] - 397s 509ms/step - loss: 0.3668 - acc: 0.8857 - val_loss: 0.4137 - val_acc: 0.8908

Epoch 00180: val_acc did not improve from 0.89353
Epoch 181/193
781/781 [==============================] - 397s 508ms/step - loss: 0.3649 - acc: 0.8860 - val_loss: 0.4080 - val_acc: 0.8922

Epoch 00181: val_acc did not improve from 0.89353
Epoch 182/193
781/781 [==============================] - 397s 508ms/step - loss: 0.3655 - acc: 0.8844 - val_loss: 0.4072 - val_acc: 0.8934

Epoch 00182: val_acc did not improve from 0.89353
Epoch 183/193
781/781 [==============================] - 392s 502ms/step - loss: 0.3672 - acc: 0.8848 - val_loss: 0.4063 - val_acc: 0.8937

Epoch 00183: val_acc improved from 0.89353 to 0.89373, saving model to weights.183-0.89.hdf5
Epoch 184/193
781/781

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.0006, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//32,
                    epochs= 199,
                    verbose=1,
                    initial_epoch = 193,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//32)

Epoch 194/199


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


1562/1562 [==============================] - 826s 529ms/step - loss: 0.3629 - acc: 0.8861 - val_loss: 0.3962 - val_acc: 0.8945

Epoch 00194: val_acc did not improve from 0.89663
Epoch 195/199
1562/1562 [==============================] - 785s 503ms/step - loss: 0.3605 - acc: 0.8860 - val_loss: 0.3919 - val_acc: 0.8952

Epoch 00195: val_acc did not improve from 0.89663
Epoch 196/199
1562/1562 [==============================] - 787s 504ms/step - loss: 0.3569 - acc: 0.8875 - val_loss: 0.3995 - val_acc: 0.8945

Epoch 00196: val_acc did not improve from 0.89663
Epoch 197/199
1562/1562 [==============================] - 791s 507ms/step - loss: 0.3568 - acc: 0.8876 - val_loss: 0.3964 - val_acc: 0.8952

Epoch 00197: val_acc did not improve from 0.89663
Epoch 198/199
1562/1562 [==============================] - 783s 501ms/step - loss: 0.3559 - acc: 0.8885 - val_loss: 0.3905 - val_acc: 0.8977

Epoch 00198: val_acc improved from 0.89663 to 0.89769, saving model to weights.198-0.90.hdf5
Epoch 199/1

In [0]:
model.load_weights('weights.198-0.90.hdf5')

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.0003, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 209,
                    verbose=1,
                    initial_epoch = 199,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 200/209


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 441s 564ms/step - loss: 0.3528 - acc: 0.8888 - val_loss: 0.3903 - val_acc: 0.8981

Epoch 00200: val_acc improved from 0.89769 to 0.89814, saving model to weights.200-0.90.hdf5
Epoch 201/209
649/781 [=======================>......] - ETA: 1:02 - loss: 0.3542 - acc: 0.8889

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.00035, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.load_weights('weights.200-0.90.hdf5')

In [26]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//64,
                    epochs= 209,
                    verbose=1,
                    initial_epoch = 201,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//64)

Epoch 202/209


/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:799: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:807: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


781/781 [==============================] - 403s 516ms/step - loss: 0.3543 - acc: 0.8892 - val_loss: 0.3938 - val_acc: 0.8965

Epoch 00202: val_acc improved from -inf to 0.89653, saving model to weights.202-0.90.hdf5
Epoch 203/209
781/781 [==============================] - 376s 482ms/step - loss: 0.3533 - acc: 0.8874 - val_loss: 0.3912 - val_acc: 0.8969

Epoch 00203: val_acc improved from 0.89653 to 0.89694, saving model to weights.203-0.90.hdf5
Epoch 204/209
781/781 [==============================] - 376s 482ms/step - loss: 0.3553 - acc: 0.8885 - val_loss: 0.3921 - val_acc: 0.8958

Epoch 00204: val_acc did not improve from 0.89694
Epoch 205/209
781/781 [==============================] - 377s 482ms/step - loss: 0.3532 - acc: 0.8890 - val_loss: 0.3872 - val_acc: 0.8975

Epoch 00205: val_acc improved from 0.89694 to 0.89754, saving model to weights.205-0.90.hdf5
Epoch 206/209
781/781 [==============================] - 376s 482ms/step - loss: 0.3557 - acc: 0.8866 - val_loss: 0.3896 - val_a

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.00015, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [29]:
model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=219,
                    verbose=1,
                    initial_epoch=209,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 210/219
50000/50000 [==============================] - 394s 8ms/step - loss: 0.2803 - acc: 0.9136 - val_loss: 0.3525 - val_acc: 0.9013

Epoch 00210: val_acc improved from 0.89934 to 0.90130, saving model to weights.210-0.90.hdf5
Epoch 211/219
50000/50000 [==============================] - 365s 7ms/step - loss: 0.2745 - acc: 0.9165 - val_loss: 0.3494 - val_acc: 0.9024

Epoch 00211: val_acc improved from 0.90130 to 0.90240, saving model to weights.211-0.90.hdf5
Epoch 212/219
50000/50000 [==============================] - 365s 7ms/step - loss: 0.2664 - acc: 0.9192 - val_loss: 0.3513 - val_acc: 0.9029

Epoch 00212: val_acc improved from 0.90240 to 0.90290, saving model to weights.212-0.90.hdf5
Epoch 213/219
50000/50000 [==============================] - 366s 7ms/step - loss: 0.2638 - acc: 0.9205 - val_loss: 0.3517 - val_acc: 0.9028

Epoch 00213: val_acc did not improve from 0.90290
Epoch 214/219
50000/50000 [==========================

In [0]:
datagen = ImageDataGenerator(horizontal_flip=True)

In [0]:
data_generator = datagen.flow(x_train, y_train, batch_size=64)
test_generator = ImageDataGenerator().flow(x_test, y_test, batch_size=64)

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.0001, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

*To speedup training the batch size is increased to 128 and this also boosted accuracy.*

In [34]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//128,
                    epochs= 229,
                    verbose=1,
                    initial_epoch = 219,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//128)

Epoch 220/229
390/390 [==============================] - 186s 476ms/step - loss: 0.2590 - acc: 0.9226 - val_loss: 0.3449 - val_acc: 0.9052

Epoch 00220: val_acc improved from 0.90470 to 0.90525, saving model to weights.220-0.91.hdf5
Epoch 221/229
390/390 [==============================] - 184s 471ms/step - loss: 0.2635 - acc: 0.9200 - val_loss: 0.3452 - val_acc: 0.9050

Epoch 00221: val_acc did not improve from 0.90525
Epoch 222/229
390/390 [==============================] - 183s 470ms/step - loss: 0.2543 - acc: 0.9241 - val_loss: 0.3447 - val_acc: 0.9044

Epoch 00222: val_acc did not improve from 0.90525
Epoch 223/229
390/390 [==============================] - 183s 469ms/step - loss: 0.2616 - acc: 0.9207 - val_loss: 0.3438 - val_acc: 0.9050

Epoch 00223: val_acc did not improve from 0.90525
Epoch 224/229
390/390 [==============================] - 183s 469ms/step - loss: 0.2663 - acc: 0.9199 - val_loss: 0.3444 - val_acc: 0.9050

Epoch 00224: val_acc did not improve from 0.90525
Epoch 2

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.00012, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [39]:
model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=234,
                    verbose=1,
                    initial_epoch=229,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 230/234
50000/50000 [==============================] - 335s 7ms/step - loss: 0.2485 - acc: 0.9245 - val_loss: 0.3496 - val_acc: 0.9049

Epoch 00230: val_acc did not improve from 0.90685
Epoch 231/234
50000/50000 [==============================] - 328s 7ms/step - loss: 0.2463 - acc: 0.9256 - val_loss: 0.3508 - val_acc: 0.9044

Epoch 00231: val_acc did not improve from 0.90685
Epoch 232/234
50000/50000 [==============================] - 328s 7ms/step - loss: 0.2479 - acc: 0.9253 - val_loss: 0.3511 - val_acc: 0.9044

Epoch 00232: val_acc did not improve from 0.90685
Epoch 233/234
50000/50000 [==============================] - 328s 7ms/step - loss: 0.2443 - acc: 0.9275 - val_loss: 0.3495 - val_acc: 0.9056

Epoch 00233: val_acc did not improve from 0.90685
Epoch 234/234
50000/50000 [==============================] - 329s 7ms/step - loss: 0.2445 - acc: 0.9255 - val_loss: 0.3502 - val_acc: 0.9052

Epoch 00234: val_acc did not improve fro

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.000095, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit_generator(data_generator,
                   steps_per_epoch=50000//128,
                    epochs= 239,
                    verbose=1,
                    initial_epoch = 234,
                    callbacks=callbacks_list,
                    validation_data=test_generator,validation_steps=10000//128)

In [0]:
# determine Loss function and Optimizer
sgd = SGD(lr=0.00005, decay=1e-4, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=250,
                    verbose=1,
                    initial_epoch=239,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

In [0]:
from google.colab import files

files.download('DNST_model.h5')